<a href="https://colab.research.google.com/github/dgl2000/comp559_ml_with_graphs/blob/main/comp_559_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COMP 559 HW 3
@author: Gaole Dai (gd25)



## Question 2

In [ ]:
from sklearn.datasets import load_wine
from sklearn.semi_supervised import LabelPropagation
from sklearn.metrics import accuracy_score
import numpy as np

# Load the wine dataset
X, y = load_wine(return_X_y=True)
np.random.seed(2)

# Randomly select 30% of the data to be unlabelled
mask = np.random.choice(len(y), size=int(len(y) * 0.3), replace=False)
y_missing = np.copy(y)
y_missing[mask] = -1

# Train the label propagation model
lp_model = LabelPropagation(gamma=.25).fit(X, y_missing)

# Evaluate the accuracy of the model
y_true = y[mask]
y_pred = lp_model.transduction_[mask]
accuracy = accuracy_score(y_true, y_pred)

print(f"Accuracy for random selection: {accuracy:.2f}")

Accuracy for random selection: 0.70


In [ ]:
from sklearn.cluster import KMeans
from sklearn.semi_supervised import LabelPropagation
from sklearn.metrics import accuracy_score
import numpy as np

# Load the wine dataset
X, y = load_wine(return_X_y=True)

# Train a KMeans clustering model to obtain cluster labels
n_clusters = int(len(y) * 0.7)
kmeans_model = KMeans(n_clusters=n_clusters).fit(X)
cluster_labels = list(kmeans_model.labels_)

# Randomly select 30% of the data to be unlabelled
indices = []
num_elements_to_remove = int(len(y) * 0.7)

indices = [cluster_labels.index(i) for i in range(num_elements_to_remove)]
mask = [i for i in range(len(y)) if i not in indices]

# Set the unlabelled data to -1
y_missing = np.copy(y)
y_missing[mask] = -1

# Train the label propagation model
lp_model = LabelPropagation(gamma=.25)
lp_model.fit(X, y_missing)


# Evaluate the accuracy of the model
y_true = y[mask]
y_pred = lp_model.transduction_[mask]
accuracy = accuracy_score(y_true, y_pred)

print(f"Accuracy for K-means clustering selection: {accuracy:.2f}")


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Accuracy for K-means clustering selection: 0.80


## Question 3

In [ ]:
!pip install networkx==2.8.6


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.2 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.0
    Uninstalling networkx-3.0:
      Successfully uninstalled networkx-3.0


In [ ]:
import networkx as nx
import csv

# Read the edgelist from file
G = nx.read_edgelist("cora.cites")

# Initialize dictionaries for node labels and features, and a dictionary for unique labels
node_labels = {}
node_features = {}
unique_labels = {}

# Open the file containing node information
with open('cora.content') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')

    # Loop through each row in the file
    for row in reader:

        # If the label is not already in the unique_labels dictionary, add it
        label = row[-1]
        if label not in unique_labels:
            unique_labels[label] = len(unique_labels)

        # Add the label and features for the current node to the respective dictionaries
        node_id = row[0]
        node_labels[node_id] = unique_labels[label]
        node_features[node_id] = row[1:-1]

In [ ]:
import numpy as np

# Get the connected components of the graph and obtain the largest connected component
connected_components = [G.subgraph(c) for c in nx.connected_components(G)]
largest_component = max(connected_components, key=len)

# Get the nodes in the largest connected component
largest_nodes = largest_component.nodes()

# Get the labels and features for each node in the largest connected component
largest_labels = np.array([node_labels[node] for node in largest_nodes])
largest_features = np.array([node_features[node] for node in largest_nodes])


In [ ]:
from sklearn.cluster import SpectralClustering
from sklearn.metrics.cluster import adjusted_rand_score

# Load the graph from the adjacency matrix
adj_matrix = nx.adjacency_matrix(largest_component)

# Define a range of values to try for the number of clusters
n_clusters_range = range(2, 40)

# Store the results in lists
ARI_list = []
labels_list = []

# Loop over the range of values for the number of clusters
for n_clusters in n_clusters_range:
    # Apply spectral clustering to the adjacency matrix
    spectral_cluster = SpectralClustering(n_clusters=n_clusters, affinity='precomputed', assign_labels='cluster_qr', n_neighbors=15, random_state=5).fit(adj_matrix)
    # Get the predicted labels
    predict_label = spectral_cluster.labels_
    
    # Compute the adjusted Rand index between the predicted labels and the true labels
    ARI = adjusted_rand_score(largest_labels, predict_label)
    
    # Append the ARI and predicted labels to the corresponding lists
    ARI_list.append(ARI)
    labels_list.append(predict_label)
    
# Find the index of the maximum ARI
max_ARI_idx = np.argmax(ARI_list)

# Get the number of clusters and predicted labels corresponding to the maximum ARI
best_n_clusters = n_clusters_range[max_ARI_idx]
best_predict_label = labels_list[max_ARI_idx]

# Print the results
print(f"Best number of clusters: {best_n_clusters}")
print(f"Adjusted Rand index: {ARI_list[max_ARI_idx]}")


<ipython-input-6-586a8ec0bd79>:5: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj_matrix = nx.adjacency_matrix(largest_component)


Best number of clusters: 30
Adjusted Rand index: 0.33940973934500496


In [ ]:
from sklearn.cluster import SpectralClustering
from sklearn.metrics.cluster import adjusted_rand_score

# Define a range of values to try for the number of clusters
n_clusters_range = range(2, 40)

# Store the results in lists
ARI_list = []
labels_list = []

# Loop over the range of values for the number of clusters
for n_clusters in n_clusters_range:
    # Apply spectral clustering to the adjacency matrix
    spectral_cluster = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', n_init =100 , n_neighbors =5, random_state =0).fit(largest_features)
    # Get the predicted labels
    predict_label = spectral_cluster.labels_
    
    # Compute the adjusted Rand index between the predicted labels and the true labels
    ARI = adjusted_rand_score(largest_labels, predict_label)
    
    # Append the ARI and predicted labels to the corresponding lists
    ARI_list.append(ARI)
    labels_list.append(predict_label)
    
# Find the index of the maximum ARI
max_ARI_idx = np.argmax(ARI_list)

# Get the number of clusters and predicted labels corresponding to the maximum ARI
best_n_clusters = n_clusters_range[max_ARI_idx]
best_predict_label = labels_list[max_ARI_idx]

# Print the results
print(f"Best number of clusters: {best_n_clusters}")
print(f"Adjusted Rand index: {ARI_list[max_ARI_idx]}")


Best number of clusters: 9
Adjusted Rand index: 0.0985192999739339
